Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    To Do:

    Figure out script to iterate through and cleanup all CSVs

    Store data from all CSVs

    Aggregate data from *all* CSVs into one dataframe
    

In [10]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv',index_col=False)
services = services.astype({'date':'datetime64'})
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services = services.astype({'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float64',
                'line' : 'category',
                'type' : 'category'})

services = services[services['delay_minutes'].notnull() & services['delay_minutes'] >= 1.0]
services

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,line,type
1,2018-03-01,3805,2.0,New York Penn Station,105,Secaucus Upper Lvl,38187,2018-03-02 01:31:00,2018-03-02 01:31:08,0.133333,Northeast Corrdr,NJ Transit
2,2018-03-01,3805,3.0,Secaucus Upper Lvl,38187,Newark Penn Station,107,2018-03-02 01:40:00,2018-03-02 01:40:07,0.116667,Northeast Corrdr,NJ Transit
3,2018-03-01,3805,4.0,Newark Penn Station,107,Newark Airport,37953,2018-03-02 01:45:00,2018-03-02 01:45:10,0.166667,Northeast Corrdr,NJ Transit
4,2018-03-01,3805,5.0,Newark Airport,37953,North Elizabeth,109,2018-03-02 01:49:00,2018-03-02 01:49:10,0.166667,Northeast Corrdr,NJ Transit
5,2018-03-01,3805,6.0,North Elizabeth,109,Elizabeth,41,2018-03-02 01:52:00,2018-03-02 01:52:01,0.016667,Northeast Corrdr,NJ Transit
...,...,...,...,...,...,...,...,...,...,...,...,...
256501,2018-03-31,1875,12.0,Ho-Ho-Kus,64,Waldwick,151,2018-03-31 17:07:00,2018-03-31 17:08:00,1.000000,Bergen Co. Line,NJ Transit
256503,2018-03-31,0534,2.0,Bay Street,14,Glen Ridge,50,2018-03-31 19:02:00,2018-03-31 19:05:07,3.116667,Montclair-Boonton,NJ Transit
256504,2018-03-31,0534,3.0,Glen Ridge,50,Bloomfield,19,2018-03-31 19:05:00,2018-03-31 19:08:06,3.100000,Montclair-Boonton,NJ Transit
256505,2018-03-31,0534,4.0,Bloomfield,19,Watsessing Avenue,154,2018-03-31 19:07:00,2018-03-31 19:10:11,3.183333,Montclair-Boonton,NJ Transit


In [11]:
 #create new dataframe with data broken down by line

njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))


performance_by_line = pd.DataFrame({'Longest Delay (minutes)': max_delays.values,
                                   'Average Delay (minutes)': avg_delays.values,
                                   'Date of Longest Delay' : dates.values},
                                    index=njt_lines)

performance_by_line

,Longest Delay (minutes),Average Delay (minutes),Date of Longest Delay
Northeast Corrdr,91.083,5.055,2018-03-12
No Jersey Coast,100.267,4.467,2018-03-01
Main Line,86.000,3.950,2018-03-07
Morristown Line,100.283,4.322,2018-03-02
Gladstone Branch,99.083,4.114,2018-03-02
Raritan Valley,94.000,3.103,2018-03-07
Bergen Co. Line,79.000,3.385,2018-03-07
Atl. City Line,100.217,5.159,2018-03-07
Montclair-Boonton,111.183,3.859,2018-03-07
Princeton Shuttle,4.467,0.196,2018-03-04


Visualization of On-Time Performance by Rail Line

    notes: Princeton Shuttle is outlier considering it's short trip distance

In [ ]:
ax = performance_by_line.plot.bar(y='Longest Delay (minutes)')
ax

Breakdown by day of the week

In [13]:
performance_by_line['Date of Longest Delay'].apply(pd.Timestamp.dayofweek)

TypeError: 'getset_descriptor' object is not callable